In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


data = pd.read_csv("SolarPrediction.csv")

# Drop unnecessary columns
data.drop(['Date', 'Time'], axis=1, inplace=True)

# Separate features and target variable
X = data.drop('Radiation', axis=1)
y = data['Radiation']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [2]:
from sklearn.neural_network import MLPRegressor

def create_ann():
    # Define the neural network structure
    ann = MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=200, random_state=42)
    return ann


In [3]:
import numpy as np
from tqdm import tqdm
from sklearn.exceptions import ConvergenceWarning
import warnings

# Suppress convergence warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Define EO parameters
num_candidates = 30
max_iterations = 100
alpha = 1.0  # Control parameter for exploration
beta = 1.5   # Control parameter for exploitation

# Initialize candidates (particles)
candidates = np.random.uniform(-1, 1, (num_candidates, X_train.shape[1]))

# Define the fitness function
def fitness_function(candidate):
    ann = create_ann()
    ann.fit(X_train, y_train)
    predictions = ann.predict(X_test)
    mse = np.mean((predictions - y_test) ** 2)
    return mse

# Main EO loop with progress bar
for iteration in tqdm(range(max_iterations), desc="EO Training"):
    fitness_values = np.array([fitness_function(candidate) for candidate in candidates])
    
    # Determine the best candidates (equilibrium pool)
    best_indices = np.argsort(fitness_values)[:4]  # Selecting the top 4 candidates for the equilibrium pool
    equilibrium_pool = candidates[best_indices]
    
    # Update each candidate's position
    for i in range(num_candidates):
        # Randomly select an equilibrium candidate
        eq_candidate = equilibrium_pool[np.random.randint(0, equilibrium_pool.shape[0])]
        
        # Calculate the update using mass balance and energy minimization principles
        r1, r2, r3, r4 = np.random.rand(4)
        F = r1 * np.sign(r2 - 0.5) * (np.exp(-iteration / max_iterations) - 1)
        G = r3 * np.sign(r4 - 0.5) * (eq_candidate - candidates[i])
        candidates[i] = eq_candidate + F * (alpha * G + beta * (eq_candidate - candidates[i]))
        
        # Ensure the candidates' positions are within bounds
        candidates[i] = np.clip(candidates[i], -1, 1)

# Select the best candidate
best_candidate_idx = np.argmin(fitness_values)
best_candidate = candidates[best_candidate_idx]

# Train the final ANN with the best parameters
final_ann = create_ann()
final_ann.fit(X_train, y_train)

# Evaluate the final model
final_predictions = final_ann.predict(X_test)
final_mse = np.mean((final_predictions - y_test) ** 2)
print("Final MSE:", final_mse)

SOSA Training:   0%|          | 0/100 [00:00<?, ?it/s]

SOSA Training:   1%|          | 1/100 [02:58<4:54:36, 178.55s/it]